In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sympy import sieve
import time

In [61]:
path = 'e:/data/kaggle/santa2/'

nodes = pd.read_csv(path + 'cities.csv')
nodes = nodes.iloc[:,1:].values
n = len(nodes)
prime_ = sieve.primerange(0,n)
prime = set([_ for _ in prime_])

In [109]:
def score(solution,raw=False):
    s = nodes[solution[:-1]]
    e = nodes[solution[1:]]
    out = np.sqrt(((s-e)**2).sum(axis=1))
    if raw:
        return out
    isten = (np.arange(n)+1)%10==0
    notprime = ~np.isin(solution[:-1],prime)
    isextra = (isten&notprime)*0.1
    out += out*isextra
    return out

def distance(coor1,coor2):
    return np.sqrt(((coor1-coor2)**2).sum(axis=-1))


In [170]:
solution = pd.read_csv(path+'base.csv').values[:,0]
score(solution,True).sum()

1519610.2239609738

In [167]:
solution = np.arange(n)
solution = np.append(solution,[0])

In [54]:
coor = nodes[solution]
point1,point2 = coor[:2]
distance(point1,point2)

10.230012592801549

In [171]:
start = time.time()

#every iter
coor = nodes[solution]
dist = score(solution,True)
neighbor = pd.DataFrame({'d':dist}).d.rolling(2,center=True).sum().values

left = 1

better = 0

cur = np.append([0],distance(coor[0],coor))
nex = np.append([0],distance(coor[1],coor))

for left in range(1,19000):
#every left
    pre_coor, left_coor, next_coor = coor[left-1:left+2]
    adj_dist = distance(left_coor, next_coor)

    ##old cost
    left_nb = neighbor[left]
    old_cost = neighbor[left:] + left_nb
    old_cost[0] = 0
    old_cost[1] -= adj_dist

    ##new cost
    pre = cur[1:]
    
    cur = nex[1:]
    
    nex = distance(next_coor,coor[left-1:])

    cur_1 = cur[:-2]
    cur_2 = cur[2:]
    
    new_cost = pre[1:-1] + nex[1:-1] + cur_1 + cur_2
    new_cost[0] = 0
    new_cost[1] += adj_dist
    
    save = (old_cost - new_cost).max()
    if save>0:
        better+=save
        #print(left,save.max())
print(time.time()-start)
print(better)

179.525000333786
10.660509027968859


In [143]:
np.append([0],distance(coor[1],coor))[1:].shape

(197770,)

In [163]:
197*10/60

32.833333333333336